## Multinomial model-NB classifier
Here we used the multinomial model.
In this case we calculate the probability of the word (given ham or spam mail) from the training set as=
no. of times this word appears in all of (ham/spam) mails combined / total no. of words in (ham/spam) mails

Now for smoothing alpha was 1;

Therefore, while predicting the testing samples.
added 1 in numerator and added 4094*1 in the denominator.
Here , the significance of 4094*1 is that 4094 is the len(dictionary) and 1 is alpha.
Therefore, we are increasing the presence of each word by one, this way our probability of a word which hasnt been encountered yet will never be 0.
This is because I am returning default value as 0 if that word is not present in the ham mails.


Now for preprocessing.
The words are imported from nltk.corpus package;
This contains a dictionary of about 10k words,
This is done so that the words which are not in proper english get ignored.
Then every sample is processed by removing all the punctuations and splitting to get a list of all the words.

This helps create a dictionary of about 4094 words.



In the case of multinomial , we need the count of a word in a (spam/ham) mail and also total number of words in the (ham/spam) mails.

Since we are giving more importance to the frequency now, this should result in better accuracy than bernoulli.

#### A mail which contains the word money 10 times , will now have more probability to be judged as a spam mail.Whereas this isnt the case with bernoulli

In [3]:
#for calculation
import numpy as np
#to read csv
import pandas as pd
#to create dictionary
import nltk
from nltk.corpus import words

In [4]:
df=pd.read_csv('data.csv')
total_samples=len(df)
print('total_Samples=',total_samples)

df['Cat'] = df['Cat'].map({'spam': 1, 'ham': 0})

total_Samples= 5574

total_Samples= 5574


In [5]:
import string
X=df.iloc[:,1].values
Y=df.iloc[:,0].values

nltk.download('words')
set_words=set(words.words())
dictionary={}
for i in range(total_samples):
    ind=len(dictionary)
    for j in df.iloc[i,1].translate(str.maketrans('', '', string.punctuation)).split():
        if (j.lower() not in dictionary and j.lower() in set_words):
            dictionary[j]=ind
            ind+=1
print(dictionary)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


{'Go': 0, 'until': 1, 'point': 2, 'crazy': 3, 'Available': 4, 'only': 5, 'in': 6, 'n': 7, 'great': 8, 'world': 9, 'la': 10, 'e': 11, 'buffet': 12, 'Cine': 13, 'there': 14, 'got': 15, 'wat': 16, 'lar': 17, 'u': 18, 'Free': 107, 'entry': 20, 'a': 21, 'to': 22, 'win': 23, 'FA': 2017, 'Cup': 25, 'final': 26, 'May': 27, 'Text': 28, 'receive': 30, 'apply': 31, 'dun': 31, 'say': 32, 'so': 33, 'early': 34, 'c': 35, 'already': 36, 'then': 37, 'I': 38, 'dont': 39, 'think': 40, 'he': 41, 'goes': 42, 'around': 43, 'here': 44, 'though': 45, 'Hey': 1126, 'darling': 47, 'its': 48, 'been': 49, 'now': 50, 'and': 51, 'no': 52, 'word': 53, 'back': 54, 'Id': 1354, 'like': 56, 'some': 57, 'fun': 58, 'you': 59, 'up': 60, 'for': 61, 'it': 62, 'still': 63, 'send': 64, 'Even': 65, 'my': 66, 'brother': 67, 'is': 68, 'not': 69, 'speak': 70, 'with': 71, 'me': 72, 'They': 73, 'treat': 74, 'patent': 75, 'As': 76, 'per': 77, 'your': 78, 'request': 79, 'set': 80, 'as': 81, 'all': 82, 'Press': 660, 'copy': 84, 'WINNER

## training

In [6]:
indices=np.random.permutation(total_samples)
X=X[indices]
Y=Y[indices]

m=int(total_samples*70/100)
X_train=X[:m]
Y_train=Y[:m]

X_test=X[m:]
Y_test=Y[m:]


In [7]:

word_count_spam={}
word_count_ham={}
no_spam_words=0
no_ham_words=0
no_spam_mails=np.count_nonzero(Y_train==1)
no_ham_mails=np.count_nonzero(Y_train==0)


for i in range (len(X_train)):
    for j in X_train[i].translate(str.maketrans('', '', string.punctuation)).split():
        if(j.lower() in dictionary and Y_train[i]==1):#spam
            word_count_spam[j.lower()]=word_count_spam.get(j.lower(),0)+1
            no_spam_words+=1
        elif(j.lower() in dictionary and Y_train[i]==0):
            word_count_ham[j.lower()]=word_count_ham.get(j.lower(),0)+1
            no_ham_words+=1


## Testing

In [8]:
def conf_mat(Y_pred,Y_test):
    tp=0
    tn=0
    fp=0
    fn=0
    for i in range(len(Y_test)):
        if(Y_pred[i]==Y_test[i]):
            if(Y_test[i]==0):
                tn+=1
            else:
                tp+=1
        else:
            if(Y_test[i]==0):
                fp+=1
            else:
                fn+=1
    print('tn=',tn,'tp=',tp,'fn=',fn,'fp=',fp)
    precision=(tp/(tp+fp))
    print('precision',precision)
    recall=(tp/(tp+fn))
    print('recall',recall)
    print('f1score',((precision*recall*2)/(precision+recall)))
    print('Accuracy',((tp+tn)/(len(Y_test))))

In [9]:

pred=[0]*len(X_test)

for i in range(len(X_test)):

    ph=no_ham_mails/len(Y_train)
    ps=no_spam_mails/len(Y_train)

    for j in X_test[i].translate(str.maketrans('', '', string.punctuation)).split():
        if(j.lower() in dictionary):
            ph*=((word_count_ham.get(j.lower(),0)+1)/(no_ham_words+len(dictionary)))
            ps*=((word_count_spam.get(j.lower(),0)+1)/(no_spam_words+len(dictionary)))

    
    if(ph>ps):

        pred[i]=0
    else:

        pred[i]=1

conf_mat(pred,Y_test)


tn= 1417 tp= 208 fn= 31 fp= 17
precision 0.9244444444444444
recall 0.8702928870292888
f1score 0.896551724137931
Accuracy 0.9713090257023311


We observe an increase in accuracy from the bernoulli model.